In [3]:
import requests
import json
import pandas as pd

In [4]:
link = "https://api.rescuegroups.org/http/v2.json"

In [5]:
api_key = 'ZJJi3YFv'

In [6]:
kept_fields = [u'animalActivityLevel',"animalSizeUOM", u'animalAdoptedDate',
               u'animalAdoptionFee', u'animalAdoptionPending', u'animalAltered', u'animalAvailableDate',
               u'animalBirthdate', u'animalBreed',  u'animalColor', u'animalColorDetails', u'animalDeclawed',
               u'animalDescription', u'animalDescriptionPlain',  u'animalFound', u'animalFoundDate',  u'animalGeneralAge',
               u'animalGeneralSizePotential',  u'animalHousetrained',  u'animalID',  u'animalIndoorOutdoor',  u'animalKillDate',
               u'animalKillReason',  u'animalLocation',  u'animalLocationState', u'animalMicrochipped', u'animalMixedBreed',
               u'animalName', u'animalNeedsFoster', u'animalNewPeople',   u'animalOKWithCats', u'animalOKWithDogs', 
               u'animalOKWithKids', u'animalOrgID', u'animalPictures', u'animalPrimaryBreed', u'animalRescueID',
               u'animalSearchString', u'animalSecondaryBreed', u'animalSex', u'animalSizeCurrent', u'animalSizePotential',
               u'animalSpecialneeds', u'animalSpecialneedsDescription', u'animalStatus', u'animalStatusID', u'animalSummary',
               u'animalUpdatedDate', u'animalUptodate', u'animalUrl', u'animalVideos', u'fosterName',
               u'locationState', u'locationUrl']

In [11]:
result = requests.post(link, json = {"apikey":api_key,
    "objectType":"animals",
    "objectAction":"publicSearch",
    "search" :{
        "calcFoundRows" : "Yes",
        "resultStart": 0,
        "resultLimit": 10,
        "resultSort" : "animalID",
        "fields" : kept_fields,
        "filters": [{
            "fieldName": "animalSpecies",
            "operation": "equals",
            "criteria": "Dog",
            },
                    {
            "fieldName": "animalSizeUOM",
            "operation": "notequals",
            "criteria": "Centimeters",
            },
                    {
            "fieldName": "animalKillDate",
            "operation": "notblank",
            "criteria": "notblank",
            }
                   ]
        }})

result.json()

{u'data': {u'3202592': {u'animalActivityLevel': u'',
   u'animalAdoptedDate': u'3/10/2011',
   u'animalAdoptionFee': u'',
   u'animalAdoptionPending': u'No',
   u'animalAltered': u'',
   u'animalAvailableDate': u'',
   u'animalBirthdate': u'',
   u'animalBreed': u'Labrador Retriever (medium coat)',
   u'animalColor': u'Black',
   u'animalColorDetails': u'',
   u'animalDeclawed': u'No',
   u'animalDescription': u'<div class="rgDescription"><p><span style="font-size: 10pt;"><span style="font-family: comic sans ms,sans-serif;"><span style="color: #993366;">Rosie is a sweet girl around 3 yrs old. She was picked up as a stray and we can\'t believe no one&nbsp;called looking for this beauty! She is house trained and very calm in the house with great manners. She also rides well in the car or crated in the truck. She walks well on a leash and knows the "sit" command. She is a jumper and would need a 6 ft wood fence, she just wants to be with her people. Rosie would love to be an only dog, liv

In [73]:
from os import path

In [90]:
myhead = sorted(a_data.keys())

In [92]:
result.json()['foundRows']

1643122

In [99]:
import time

In [136]:
# failed_batches = []
batch_size = 2000

ct = 1643122
num_batches = (ct / batch_size) + 1 
start = time.time()

for i in [549, 564, 567, 570, 572, 573, 592, 777, 802, 803, 804, 805, 806, 807, 822, 823, 824]:
    if i % 10 == 0:
        end = time.time()
        print 'Up to batch %s. Last 10 took %s' % (i, end - start)
        start = end
        
    result = requests.post(link, json = {"apikey":api_key,
        "objectType":"animals",
        "objectAction":"publicSearch",
        "search" :{
            "calcFoundRows" : "Yes",
            "resultStart": i*batch_size,
            "resultLimit": batch_size,
            "resultSort" : "animalID",
            "fields" : kept_fields,
            "filters": [{
                "fieldName": "animalSpecies",
                "operation": "equals",
                "criteria": "Dog",
                },
                        {
                "fieldName": "animalSizeUOM",
                "operation": "notequals",
                "criteria": "Centimeters",
                }
                       ]
            }})
    
    if result.status_code != 200 or result.json()['status']=='error':
        failed_batches.append(result)
    else:
        mypath = path.expanduser('~/raw_dogs/%s.csv' % i)
        with open(mypath, 'w+') as f:
            spamWriter = csv.DictWriter(f, myhead, extrasaction='ignore')
            spamWriter.writeheader()
            for animal, a_data in result.json()['data'].iteritems():
                for medium in ['Pictures', 'Videos']:
                    p_sizes = [int(x.get('fileSize', 0)) for x in a_data.get('animal%s' % medium, [])]
                    a_data['animalBiggest%s' % medium] = max(p_sizes) if p_sizes else 0
                    a_data['animalNum%s' % medium] = len(p_sizes)
                    try:
                        a_data.pop('animal%s' % medium)
                    except:
                        pass
                a_data = {k:v.encode('utf8') if isinstance(v, basestring) else v for k,v in a_data.iteritems()}
                spamWriter.writerow(a_data)

Up to batch 570. Last 10 took 712.222934961


AttributeError: 'list' object has no attribute 'iteritems'

In [110]:
result.json()

{u'message': u'Unexpected system error', u'status': u'error'}

In [131]:
batches = [0,		154,		21,		266,		324,		380,		436,		499,
1,		155,		210,		267,		325,		381,		437,		5,
10,		156,		211,		268,		326,		382,		438,		50,
100,		157,		212,		269,		327,		383,		439,		500,
101,		158,		213,		27,		328,		384,		44,		501,
102,		159,		214,		270,		329,		385,		440,		502,
103,		16,		215,		271,		33,		386,		441,		503,
104,		160,		216,		272,		330,		387,		442,		51,
105,		161,		217,		273,		331,		388,		443,		52,
106,		162,		218,		274,		332,		389,		444,		53,
107,		163,		219,		275,		333,		39,		445,		54,
108,		164,		22,		276,		334,		390,		446,		55,
109,		165,		220,		277,		335,		391,		447,		56,
11,		166,		221,		278,		336,		392,		448,		57,
110,		167,		222,		279,		337,		393,		45,		58,
111,		168,		223,		28,		338,		394,		450,		59,
112,		169,		224,		280,		339,		395,		451,		6,
113,		17,		225,		281,		34,		396,		452,		60,
114,		170,		226,		282,		340,		397,		453,		61,
115,		171,		227,		283,		341,		398,		454,		62,
116,		172,		228,		284,		342,		399,		455,		63,
117,		173,		229,		285,		343,		4,		456,		64,
118,		174,		23,		286,		344,		40,		457,		65,
119,		175,		230,		287,		345,		400,		459,		66,
12,		176,		231,		288,		346,		401,		46,		67,
120,		177,		232,		289,		347,		402,		460,		68,
121,		178,		233,		29,		348,		403,		461,		69,
122,		179,		234,		290,		349,		404,		462,		7,
123,		18,		235,		291,		35,		405,		463,		70,
124,		180,		236,		292,		350,		406,		464,		71,
125,		181,		237,		293,		351,		407,		465,		72,
126,		182,		238,		294,		352,		408,		466,		73,
127,		183,		239,		295,		353,		409,		467,		74,
128,		184,		24,		296,		354,		41,		468,		75,
129,		185,		240,		297,		355,		410,		47,		76,
13,		186,		241,		298,		356,		411,		474,		77,
130,		187,		242,		299,		357,		412,		475,		78,
131,		188,		243,		3,		358,		413,		476,		79,
132,		189,		244,		30,		359,		414,		477,		8,
133,		19,		245,		300,		36,		415,		478,		80,
134,		190,		246,		301,		360,		416,		479,		81,
135,		191,		247,		302,		361,		417,		48,		82,
136,		192,		248,		303,		362,		418,		480,		83,
137,		193,		249,		304,		363,		419,		481,		84,
138,		194,		25,		305,		364,		42,		482,		85,
139,		195,		250,		306,		365,		420,		483,		86,
14,		196,		251,		307,		366,		421,		484,		87,
140,		197,		252,		308,		367,		422,		485,		88,
141,		198,		253,		309,		368,		423,		486,		89,
142,		199,		254,		31,		369,		424,		487,		9,
143,		2,		255,		310,		37,		425,		488,		90,
144,		20,		256,		311,		370,		426,		489,		91,
145,		200,		257,		312,		371,		427,		49,		92,
146,		201,		258,		313,		372,		428,		490,		93,
147,		202,		259,		314,		373,		429,		491,		94,
148,		203,		26,		315,		374,		43,		492,		95,
149,		204,		260,		316,		375,		430,		493,		96,
15,		205,		261,		317,		376,		431,		494,		97,
150,		206,		262,		32,		377,		432,		495,		98,
151,		207,		263,		321,		378,		433,		496,		99,
152,		208,		264,		322,		379,		434,		497,		sample,
153,		209,		265,		323,		38,		435,		498,]

In [132]:
failed = []
for i in xrange(503):
    if i not in batches:
        failed.append(i)
print failed

[318, 319, 320, 449, 458, 469, 470, 471, 472, 473]


In [ ]:
[549, 564, 567, 570, 572, 573, 592, 777, 802, 803, 804, 805, 806, 807, 822, 823, 824]